In [1]:
MAX_NUM_THREADS = 8

import ctypes
mkl_rt = ctypes.CDLL('libmkl_rt.so')
#print(mkl_rt.mkl_get_max_threads())
mkl_get_max_threads = mkl_rt.mkl_get_max_threads
def mkl_set_num_threads(cores):
    mkl_rt.mkl_set_num_threads(ctypes.byref(ctypes.c_int(cores)))

mkl_set_num_threads(MAX_NUM_THREADS)
print(f'Number of threads was limited to {mkl_get_max_threads()}.')

Number of threads was limited to 8.


In [2]:
import os
import time
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import numpy_tools
import loaders.eth80 as eth80
import loaders.smni_eeg as smni_eeg

from experiment_tools import temporal_sscal
from experiment_tools import kmeans_gmm_clustering_experiment
from experiment_tools import hac_clustering_experiment

import cluster.gica_contrasting

In [3]:
data_eth80_dirname = '/home/hariyuki/data/eth80/'
data_smni_dirname = '/home/hariyuki/data/eeg_smni/'

In [4]:
# load ETH80
data_eth80_path = os.path.join(data_eth80_dirname, 'eth80-cropped-close128')

image_shape = 32
Nclasses, Nobjects = 8, 10
Npixels = int(round(image_shape**2))

data, labels, classes = eth80.eth80_dataset(data_eth80_path, image_shape)
classes_reverse_dict = dict((y, x) for x, y in classes.items())
Nsamples, Nangles, _, _, Ncolors = data.shape
data = numpy_tools.reshape_np(data, [Nangles, -1, Ncolors], order='F', use_batch=True)
permutation = [0, 2, 1, 3]
data = np.transpose(data, permutation)

n = [Npixels, Nangles, Ncolors]

eth80_dataset = {
    'data': data,
    'n': n,
    'labels': labels,
    'classes': classes,
    'classes_reverse_dict': classes_reverse_dict
}

In [5]:
# load SMNI EEG
data_smni_path = os.path.join(data_smni_dirname, 'smni_eeg_processed.npz')
df = np.load(data_smni_path)
data, labels = df['data'], df['labels']

Nsubjects, Nchannels, Ntime, Nconditions = data.shape
permutation = [0, 2, 1, 3]
data = np.transpose(data, permutation)
n = [Ntime, Nchannels, Nconditions]

smni_dataset = {
    'data': data,
    'n': n,
    'labels': labels
}

In [6]:
datasets = {
    'eth80': eth80_dataset,
    'smni': smni_dataset
}
dataset_names = list(datasets.keys())

## GICA contrasting

In [7]:
commonRanks = 1+np.arange(5)
individualRanks = 1+np.arange(5)

In [ ]:
save_results_dirname = '../results/clustering/'
os.makedirs(save_results_dirname, exist_ok=True)

save_results_filename_ac = 'gica_contrasted_ac'
save_results_filename_kg = 'gica_contrasted_kg'

Ntrials = 20
Ntrials_gica = 10
np_random_seed = 720

np_random_state_gica = 800
np.random.seed(np_random_state_gica)
random_states_gica = np.random.randint(1, 1000, Ntrials_gica)
   
def preprocess_gica(dataset, **kwargs):
    assert 'commonRank' in kwargs
    commonRank = kwargs['commonRank']
    assert 'individualRank' in kwargs
    individualRank = kwargs['individualRank']
    assert 'random_state' in kwargs
    random_state = kwargs['random_state']
    if 'sscal' in kwargs:
        sscal = kwargs['sscal']
    else:
        sscal = False
    labels = dataset['labels']
    n = dataset['n']
    contrasting = cluster.gica_contrasting.GICAContrast(
        individualRank=individualRank,
        commonRank=commonRank,
        shapeObject=[n[0], int(np.prod(n[1:]))],
        maxitnum=100,
        epsilon=1e-5,
        feature_extractor='fast_ica',
        random_state=random_state
    )
    T = contrasting.fit_transform(dataset['data'].copy())
    if sscal:
        T = T - np.mean(T, axis=1, keepdims=True)
        T /= np.std(T, axis=1, keepdims=True, ddof=1)
    return T, labels

result_ac, result_kg = [], []
affinity_names_old, linkage_old, dataset_names_old, n_clusters_old = None, None, None, None
for k_comr in range(len(commonRanks)):
    commonRank = commonRanks[k_comr]
    result_ac_crank, result_kg_crank = [], []
    for k_indr in range(len(individualRanks)):
        individualRank = individualRanks[k_indr]
        print(f'\t GICA: crank={commonRank}, irank={individualRank}')
        result_ac_crank_irank, result_kg_crank_irank = [], []
        for k_trial in range(Ntrials_gica):
            preprocess = {
                'eth80': lambda x: preprocess_gica(
                    x,
                    commonRank=commonRank,
                    individualRank=individualRank,
                    random_state=random_states_gica[k_trial]
                ),
                'smni': lambda x: preprocess_gica(
                    x,
                    commonRank=commonRank,
                    individualRank=individualRank,
                    random_state=random_states_gica[k_trial],
                    sscal=False
                )
            }
            result, affinity_names, linkage, dataset_names, n_clusters, df = hac_clustering_experiment(
                datasets,
                save_results_path=None,#save_results_path,
                preprocess=preprocess,
                verbose=True,
                return_datasets=True
            )
            if not ((k_comr == 0) and (k_indr == 0)):
                assert np.all([dataset_names[l] == dataset_names_old[l] for l in range(len(datasets))])
            dataset_names_old = dataset_names
            result_ac_crank_irank.append(result)

            result, dataset_names, random_states, clust_alg_names, n_clusters = (
                kmeans_gmm_clustering_experiment(
                    df,#datasets,
                    save_results_path=None,#save_results_path,
                    preprocess=None,#preprocess,
                    Ntrials=Ntrials,
                    np_random_seed=np_random_seed,
                    verbose=True
                )
            )
            assert dataset_names == dataset_names_old
            if not ((k_comr == 0) and (k_indr == 0)):
                assert np.all([random_states_old[l] == random_states[l] for l in range(Ntrials)])
            random_states_old = random_states
            result_kg_crank_irank.append(result)
            
            np.savez_compressed(
                'gica_contrasted_clustering_temp',
                result_ac=result_ac,
                result_kg=result_kg,
                result_ac_crank=result_ac_crank,
                result_kg_crank=result_kg_crank,
                result_ac_crank_irank=result_ac_crank_irank,
                result_kg_crank_irank=result_kg_crank_irank,
            )
            
        result_ac_crank.append(result_ac_crank_irank)
        result_kg_crank.append(result_kg_crank_irank)
    result_ac.append(result_ac_crank)
    result_kg.append(result_kg_crank)
result_ac, result_kg = np.array(result_ac), np.array(result_kg)
np.savez_compressed(
    'gica_contrasted_clustering',
    result_ac=result_ac,
    result_kg=result_kg,
    random_states=random_states,
    clust_alg_names=clust_alg_names,
    Ntrials=Ntrials,
    np_random_seed=np_random_seed,
    affinity_names=affinity_names,
    linkage=linkage,
    dataset_names=dataset_names,
    random_states_gica=random_states_gica,
    np_random_state_gica=np_random_state_gica
)

	 GICA: crank=1, irank=1
			 eth80
		 Affinity: l1
	 Linkage: complete
ARI=0.401 AMI=0.595 FMI=0.488 
	 Linkage: average
ARI=0.544 AMI=0.764 FMI=0.638 
		 Affinity: l2
	 Linkage: complete
ARI=0.508 AMI=0.700 FMI=0.589 
	 Linkage: average
ARI=0.400 AMI=0.666 FMI=0.538 
		 Affinity: cosine
	 Linkage: complete
ARI=0.534 AMI=0.720 FMI=0.603 
	 Linkage: average
ARI=0.561 AMI=0.726 FMI=0.630 
		 Affinity: canberra
	 Linkage: complete
ARI=0.431 AMI=0.613 FMI=0.510 
	 Linkage: average
ARI=0.502 AMI=0.736 FMI=0.602 
		 Affinity: correlation
	 Linkage: complete
ARI=0.534 AMI=0.720 FMI=0.603 
	 Linkage: average
ARI=0.561 AMI=0.726 FMI=0.630 
		 Affinity: rbf
	 Linkage: complete
ARI=0.016 AMI=0.032 FMI=0.327 
	 Linkage: average
ARI=0.016 AMI=0.032 FMI=0.327 
			 smni
		 Affinity: l1
	 Linkage: complete
ARI=0.013 AMI=0.009 FMI=0.729 
	 Linkage: average
ARI=0.013 AMI=0.009 FMI=0.729 
		 Affinity: l2
	 Linkage: complete
ARI=0.013 AMI=0.009 FMI=0.729 
	 Linkage: average
ARI=0.013 AMI=0.009 FMI=0.729 


In [ ]:
from td.utils import reshape, prodTenMat

import gica

maxitnum = 200
epsilon = 1e-8
verbose = True

_maxInnerIt = 15
_tolRes = 1e-8
_tolGrad = 1e-8
_tolSwamp = 1e-8

individualRank = 4
commonRank = 2
random_state = 576

shape = [-1] + list(n)
T = reshape(data, shape)
Nsubj = len(T)
np.random.seed(random_state)

parameters = {}
#parameters['eps'] = epsilon
parameters['maxitnum'] = maxitnum
parameters['R_com'] = commonRank
parameters['rPCA_com'] = commonRank
parameters['rPCA_ind'] = individualRank
parameters['random_state'] = random_state
individualRank_isint = isinstance(individualRank, (int, np.integer))
if individualRank_isint:
    parameters['rPCA_ind'] = [individualRank]*Nsubj
else:
    assert len(parameters['rPCA_ind']) == Nsubj, "Inividual ranks must be specified for each subject"
parameters['feature_extractor'] = 'fast_ica'
groupDict = gica.gica(T, **parameters)

G = np.dot(groupDict['ica']['sources'], groupDict['ica']['mixing'].T)
G = np.dot(G, groupDict['gpca']['transform'].T) + groupDict['gpca']['mean']
contrastedT = T.astype(np.double)
groupT = np.zeros(T.shape)
ind = 0
for p in range(Nsubj):
    offset = parameters['rPCA_ind'][p]
    contrastedT[p, :, :] -= np.dot(G[:, ind:ind+offset], groupDict['ipca']['transform'][p].T)
    contrastedT[p, :, :] -= groupDict['ipca']['mean'][p]
    groupT[p, :, :] += np.dot(G[:, ind:ind+offset], groupDict['ipca']['transform'][p].T)
    groupT[p, :, :] += groupDict['ipca']['mean'][p]
    ind += offset
contrastedT = reshape(contrastedT, [-1, Npixels, Nangles, Ncolors])
groupT = reshape(groupT, [-1, Npixels, Nangles, Ncolors])


In [ ]:
np.savez_compressed(
    'example_gica_eth80.npz',
    groupT=groupT,
    contrastedT=contrastedT,
    individualRank=individualRank,
    commonRank=commonRank,
    random_state=random_state
)

In [ ]:
from PIL import Image

nframe = 25
nobj = 71

def get_image(tensor, nobj, nframe):
    image = tensor[nobj, :, nframe, :]
    image = image - image.min()
    image = image / image.max()
    image = reshape(image, [32, 32, 3])
    image *= 255.
    return image.astype(np.uint8)

image = get_image(data, nobj, nframe)
plt.imshow(image)
Image.fromarray(image).save('original.jpg')
plt.show()
image = get_image(groupT, nobj, nframe)
plt.imshow(image)
Image.fromarray(image).save('gica_common.jpg')
plt.show()
image = get_image(contrastedT, nobj, nframe)
plt.imshow(image)
Image.fromarray(image).save('gica_individual.jpg')
plt.show()